In [28]:
# Induction motor calculations and assumptions
import numpy as np


#Assumptions
Poles = 4
V_Dc = 96                  # input Dc voltage in V
m = 0.9                     # modulation index,usually less than 1 
OP_Pwr = 3000               # output power of motor - 3Kw 
Power_fac = 0.8             # power factor - cos(phi)
n_eff = 0.75                # typical efficiency of  1-5 hp motor
K_ws = 0.95                 # stator winding factor kw = kp * kd * ks
freq_hz = 120 
#Dia = 0.07                  # Inner diameter of stator diameter in m L/tau_p = 1.5
#Len = 0.08                  # length in m
ac = 15000                  # specific electric loading
tau_s1 = 0.0091             # slot pitch 15mm - 20mm
del_s = 4                   # current density in stator conductor 4A/mm2
del_r = 5                 # cuurent density in rotor bar 5-7 A/mm2
mu_0 = (4*np.pi * 10**(-7)) #permeability of free space
B_y = 1.3                   # flux density in yoke; 1.3 to 1.5 T
N=120*freq_hz/Poles


def Calc_mag_flux(poles, pow_w, freq):
    mf = (0.00145 + (0.003 / poles)) * (np.sqrt((0.08 * pow_w) / freq))
    return mf 

def Calc_mag_flux_density(poles,pwr, freq, dia, length):
    b = (poles * Calc_mag_flux(poles,pwr,freq)) / (2 * dia * length) 
    return b
def Calc_air_gap(dia, length):
    lag = 0.2 + 2 * np.sqrt(dia*length)
    return lag


#Stator Design
print("\n************************Stator Design************************")

V_Ac_llrms = round((0.8165 / np.sqrt(2)) * V_Dc * m)
V_Ac_phiRms = round((V_Ac_llrms / np.sqrt(3)))          #Vph1

print("V_Ac_llrms = ", V_Ac_llrms, "V")
print("V_Ac_phiRms = Vph1 = ", V_Ac_phiRms,"V")


I_rms = ((OP_Pwr / 1000 )* 10**(3))/( 3 *  V_Ac_phiRms * Power_fac * n_eff )    #Iph1
print("I_rms =", I_rms,"A")


C_OpCoeff = 17.35 * 10**(-5)  * ac * Power_fac * n_eff
print("C_OpCoeff = ", C_OpCoeff)

D2L=(OP_Pwr)/(C_OpCoeff*N) 
D=(D2L/1.178)**(1./3)
print("Stator inner Dia = ", D)
L=1.178*D
print("Length of stator core = ", L)




print("max mag flux = ", Calc_mag_flux(Poles, OP_Pwr, freq_hz))
print("mag flux den = ", Calc_mag_flux_density(Poles, OP_Pwr, freq_hz, D, L), "wb/m2")

#print("air gap = ", Calc_air_gap(D,Len))
Ss=24 
#Ss = (((np.pi * D) / tau_s1))
#if (S1 % 4 != 0):
 #   temp = round(S1 / 4)
  #  S1 = 4 * temp

#print("No of stator slots = ", S1)

#Ss = round(S1 / (3 * Poles),1)  #no. of slots per pole per phase
#print("no. of slots per pole per phase = ", Ss)

N_ph1 = round(V_Ac_phiRms / (4.44 * K_ws * freq_hz * Calc_mag_flux(Poles, OP_Pwr, freq_hz)))
print("No of turns per phase = ", N_ph1)

Z1 = round(6 * N_ph1)  #total no of conductors
print("total no of conductors = ", Z1)

N_c1 = (Z1 / S1)
print("No of conductors per slot = ", N_c1)

Ag=(np.pi*D*L)/Poles# Air gap area
Bg=Calc_mag_flux(Poles, OP_Pwr, freq_hz)/Ag  #Air gap flux density

# Assumptions or values to be assigned

#Stator slot design :nv, nh, dc, insul_thickness, wedge, tooth_lip, slack_toler
#Flux density in stator tooth: hs, bs_1, K_i, l_tooth,
#Estimation of iron losses in stator: piy, pit, density_of_core_material,volume_of_core_material_in_teeth, volume_of_core_material_in_yoke

#Stator slot design
#N_c1 = Nv * Nh

F_c1 = I_rms / del_s    #Cross sectional area of stator conductor
print("sectional area of stator conductor = ", F_c1, "in mm2")

#As area >5sq.mm ,rectangular conductors chosen with 2.5*5=12.5sq.mm;with insulation thickness of .2mm
Ar_st_cond=2.7*5.2   

#2 conductors per slot,one in each layer;one conductor widthwise & 2 conductors depthwise
#width of slot

b_s1 =( 1*2.7+2*.2+.9 )*.001     #Nh * dc + 2 * insul_thickness + slack_toler  i.e no of hori. cond*width of cond +slot liner+clearance
print("Width of the slot = ", b_s1)

#depth of slot
H_s1 = (2*5.2+2*1+1*.2+.5+.5+3+1.5+.9)*.001 #Nv * dc + 3 * insul_thickness + wedge + tooth_lip + slack_toler in m
print("Height of slot = ", H_s1)


b_01 = (2 / 5) * b_s1
print("Slot opening = ", b_01)

#Copper loss in stator winding
#Length of mean turns
tau_p = (np.pi * D) / Poles
L_mt = 2 * L + 2.3 * tau_p + 0.24
print("Length of mean turns = ", L_mt)

#Resistance of stator winding per phase
R_ph1 = 0.021 * 10**(-6) * (L_mt / (F_c1 * 10**(-6))) * N_ph1
print("Resistance of stator winding per phase = ", R_ph1, "ohm")

#Total copper loss in the stator winding
print("Total copper loss in stator winding = ", 3 * I_rms**2 * R_ph1, "W")
    
#Flux density in stator tooth:
D1_3_s = D + (1./3) * H_s1 * 2     #Dia of stator at 1/3rd of tooth height
print("Dia of stator at 1/3rd of tooth height = ", D1_3_s)
    
tau_s1_3_s = (np.pi * D1_3_s) / S1  #Slot pitch of stator at 1/3rd of tooth height
print("Slot pitch of stator at 1/3rd of tooth height = ", tau_s1_3_s)
    
b_1_3_s =tau_s1_3_s - .004 #tau_s1_3_s - bs_1   width of stator tooth at 1/3rd of tooth height   
print("width of stator tooth at 1/3rd of tooth height = ", b_1_3_s)

#Assuming 3 ventilating ducts with 1cm width and iron space factor .95
l_tooth=(L-3*.01) #iron length
K_i=.95
A_1_s = b_1_3_s  * l_tooth  #area of one stator tooth at 1/3rd of tooth height
print("area of one stator tooth at 1/3rd of tooth height = ", A_1_s)

A_st_s = A_1_s * (S1 / Poles)  #area of all stator teeth under one pole
print("area of all stator teeth under one pole = ", A_st_s)

B_t = Calc_mag_flux(Poles, OP_Pwr, freq_hz) / A_st_s   #mean flux density in stator teeth
print("mean flux density in stator = ", B_t)

#flux density in stator core 


#Depth (Height) of stator yoke:
hy = (phi_1_s / 2) / (B_y * K_i * l_tooth)
print("Depth (Height) of stator yoke = ", hy)

#Outer dia of IM
D0 = D + (2 * H_s1) + (2 * hy)
print("Outer dia of IM = ", D0)

#Estimation of iron losses in stator

#losses in stator teeth
A_ts=b_1_3_s * l_tooth 
V_ts =A_ts * Ss * H_s1   #volume of all teeth

W_ts =V_ts* 7800  #density of silicon steel 7800 kg/cu.m
##for stator teeth mean flux density of 8 W/kg iron loss
iron_loss_teeth =8* W_ts

#Losses in stator core
A_cs= l_tooth * .191 #d_mcs=.05 depthof stator core
V_cs=A_cs *np.pi *.05
W_cs = V_cs *7800 
## for stator core flux density of .373 ,iron loss per kg=2W
iron_loss_core=W_cs *2
iron_loss= iron_loss_teeth+iron_loss_core
windage_loss=.02* OP_Pwr #2% of power
total_noload_loss=iron_loss + windage_loss
print("Total iron losses = ", iron_loss)
    
i_w= total_noload_loss/(3*V_Ac_phiRms)

##i_m calculation continued later
#Rotor Design
print("\n************************Rotor Design************************")

D_r=D-2*Calc_air_gap(D,L) *10**(-3)  #outer dia of rotor

##no of stator slots> no of rotor slots
##Sr not equal to Ss=24, Ss-Sr not equal to +- 3p ie +-12  ,Ss-Sr not = to +-p,+-2p,+-5p 
##ie 4,8,20,24-4,24-8,24-20 .  ,Ss-Srnot=to +-1,+-2,(+-p+-1),(+-p+-2) ie 24-1,24-2,24-5,24-6
Sr=21  ##for stator slots =24

#q2 = q1-0.17                                                 #assume; for 24 slot stator
#S2 = round(3 * Sr * Poles)
#print("No of rotor slots  = ", Sr)

tau_r_slot=(np.pi * D_r)/Sr 

i_r =.85*I_rms


i2_bar = (0.85 * I_rms * Ss * N_c1) / Sr
print("rotor current = ", i2_bar)

#Z_2bar = S2
#print("No of rotor bars for cage motors equals No of rotor slots = ", S2)

I2_endring = (Sr * i2_bar) / (np.pi * Poles)
print("rotor endring current = ",I2_endring)

#Size of rotor conductors:
del_bar=5     #density in A/mm2
F_c2bar = (i2_bar / del_bar)    #sectional area of rotor bar
print("sectional area of rotor bar = ", F_c2bar)

#let standard conductor size 6mm * 3mm
Ar_rotor_slot=6.5*3.5   #rotor slot dimensions in sqmm

del_ring=6.5 #in A/sqmm
F_c2endring = (I2_endring / del_ring)
print("sectional area of rotor endring = ",F_c2endring)



# Assumptions or values to be assigned
    # Flux density in rotor tooth : delta, h_t2, K_i, bs_2, l_tooth
    # Rotor copper loss in cage rotor: d_2bar
    # Slip calculation: Mech_pwr_op
    # Effective air gap length: b_01, b_02 ,b_s2, b_s1
    # Estimation of magnetizing current and no load current: A_Ty, A_T2ht1, A_T2del, A_T2ht2, A_Try, K_pd1, V_ph1, 
    # Estimation of ideal blocked rotor current: R1, R2, X1, X2
    # Estimation of leakage reactance: x, dx, h1, h2, h3,h4, h5
    # Rotor slot leakage reactance: h, K_pd2
    # Overhang leakage reacance: L_0 - Length of conductor in overhang, K_s - Slot leakage factor
    # Belt or Harmonic leakage reactance: K_h1 - factor of stator, K_h2 - factor of rotor
#Rotor copper loss
l_bar=L +.05 # len of rotor bar=L+allowance for skewing+allowance b/w end rings and rotor core
Res_1_bar = 0.021  * (l_bar / F_c2bar) 
print("Resistance in one rotor bar = ", Res_1_bar)

Cu_loss_bar = Sr * Res_1_bar * i2_bar**2
print("Copper loss in all rotor bars = ", Cu_loss_bar)


d_2bar=.002  #thickness of rotor bar
Res_endring = 0.021 * (np.pi * (D_r - 2 * d_2bar)) /  F_c2endring
print("Resistance of one end ring = ", Res_endring)

Cu_loss_endring = 2 * Res_endring * I2_endring**2
print("Copper loss in end rings = ", Cu_loss_endring)

T_Cu_loss = Cu_loss_endring + Cu_loss_bar
print("total Copper loss = ", T_Cu_loss)

    
#Flux density in rotor tooth
D1_3 = D_r - (2/3 * .002) * 2    #depth of rotor slot=.002
print("Dia of rotor at 1/3rd of tooth height = ", D1_3)

tau_s1_3 = (np.pi * D1_3) / Sr
print("slot pitch of rotor at 1/3rd of tooth height = ", tau_s1_3)

b_1_3 = tau_s1_3 - .002  
print("width of tooth at 1/3rd of tooth height = ", b_1_3)

A_1 = b_1_3 * K_i * l_tooth
print("area of one stator tooth at 1/3rd of tooth height = ", A_1)

A_rt = A_1 * (Sr / Poles)
print("area of all rotor teeth under one pole = ", A_rt)

B_tr = Calc_mag_flux(Poles, OP_Pwr, freq_hz)/A_rt 
print("mean flux density = ", B_tr)



#SLip = Rotor Cu loss / Mech Power Output + Losses
#Losses = Rotor Iron Loss (Negligible) + Rotor Cu loss + F & W loss
#Slip = 0.5% for small motors; 2.5% - 3.5% for medium motors; 1% - 1.5% for large motors 


## i_m calculation continued
#Effective air gap length
tau_sg1 = (np.pi * D) / Ss
K_01 = 1 / (1 + 5 *(Calc_air_gap(D,L) *10**(-3)/ b_s1))     #carter's coefficient
K_c1 = tau_sg1 / (tau_sg1 - (K_01 * b_01))         
print("Gap Contraction factor for stator = ", K_c1)


b_s2=.02 
b_02=(2./5)*b_s2
tau_sg2 = (np.pi * (D - 2 * Calc_air_gap(D,L)) *10**(-3)) / Sr
K_02 = 1 / (1 + 5 *(Calc_air_gap(D,L) *10**(-3) / b_s2))
K_c2 = tau_sg2 / (tau_sg2 - (K_02 * b_02))
print("Gap Contraction factor for rotor = ", K_c2)

del_ag = K_c1 * K_c2 * Calc_air_gap(D,L) *10**(-3)
print("Effective air gap length = ", del_ag)

#FLUX DENSITY DISTRIBUTION
B_30deg = 1.35 * Calc_mag_flux_density(Poles, OP_Pwr, freq_hz, D, L)
print("FLUX DENSITY at 30 deg from direct axis = ", B_30deg)

#MMF REQUIRED IN AIR GAP 
mu_0= 4*np.pi *10**(-7)
A_Tdel = (1 / mu_0) * B_30deg * del_ag
print("MMF REQUIRED IN AIR GAP = ", A_Tdel)

#MMF for stator core

## for steel AT/m =400 for .35T

A_Ts_core=400* tau_p *(1./3)

#MMF for stator teeth
B_t30deg= 1.36* B_t
## AT for .762 is 1000T and length of flux path is .019
A_Ts_teeth=19

#MMF for rotor teeth
## flux density at 30deg =1.36 *.515;corres. AT/m is 700;mean len=.01
A_Tr_teeth=700*.01

##MMF for rotor core
d_cr= .01 #assume rotor core depth
A_cr= l_tooth * d_cr 
B_cr=(.0017/A_cr)
#for .18T AT/m is 400
A_Tr_core=400*tau_p*(1./3)



#ESTIMATION OF MAGNETIZING CURRENT & NO LOAD CURRENT 
A_T30 = A_Ts_core + A_Ts_teeth + A_Tdel + A_Tr_core + A_Tr_teeth
print("AT at one pole = ", A_T30 / 2)
K_pd1=.955
i_m = A_T30 * (Poles / 2 ) * (1 / (1.17 * K_pd1 * N_ph1))

i_0 = np.sqrt(i_w**2 + i_m**2)
print("No load current = ", i_0)

cos_phi0 = i_w / i_m
print("No load power factor = ", cos_phi0)



************************Stator Design************************
V_Ac_llrms =  50.0 V
V_Ac_phiRms = Vph1 =  29.0 V
I_rms = 57.4712643678 A
C_OpCoeff =  1.5615
Stator inner Dia =  0.7680281855383874
Length of stator core =  0.9047372025642203
max mag flux =  0.00311126983722
mag flux den =  0.00895505107623 wb/m2
No of turns per phase =  18.0
total no of conductors =  108.0
No of conductors per slot =  0.407547169811
sectional area of stator conductor =  14.367816092 in mm2
Width of the slot =  0.004
Height of slot =  0.019
Slot opening =  0.0016
Length of mean turns =  3.436852635754886
Resistance of stator winding per phase =  0.0904194686235 ohm
Total copper loss in stator winding =  895.951928493 W
Dia of stator at 1/3rd of tooth height =  0.7806948522050541
Slot pitch of stator at 1/3rd of tooth height =  0.009255189480689688
width of stator tooth at 1/3rd of tooth height =  0.005255189480689688
area of one stator tooth at 1/3rd of tooth height =  0.004596909745283415
area of all sta

In [29]:
D2L

0.5336748852598996